okay starting with seeting up the venv

setup pyomo
setup sppy
setup mpi4py
    req mpic++, openmpi, etc,
    brew install mpich

test the mpi4py package installation

    mpirun -n 2 python -m mpi4py mpi_one_sided_test.py

    test errors out 

     File "mpi_one_sided_test.py", line 42, in main
    assert(buff[-1] == 3.)
AssertionError
Abort(1) on node 1 (rank 1 in comm 0): application called MPI_Abort(MPI_COMM_WORLD, 1) - process 1




env MPICC=/path/to/mpicc python3 -m pip install mpi4py

/usr/local/Cellar/mpich/4.0.1

env MPICC=/usr/local/Cellar/mpich/4.0.1 python3 -m pip install mpi4py

okay this not working either.
install the whole thing from brew?


okay so the installation of mpi4py via homebrew works in the main env, but not in the venv, I guess no surprize on that.
I should have done the whole setup in a container.

okay now should I install mpi-sppy in the main env too?
okay let's move on to a simple example

In [1]:
import pyomo.environ as pyo

def build_model(yields):
    model = pyo.ConcreteModel()

    # Variables
    model.X = pyo.Var(["WHEAT", "CORN", "BEETS"], within=pyo.NonNegativeReals)
    model.Y = pyo.Var(["WHEAT", "CORN"], within=pyo.NonNegativeReals)
    model.W = pyo.Var(
        ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"],
        within=pyo.NonNegativeReals,
    )

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST = 238 * model.Y["WHEAT"] + 210 * model.Y["CORN"]
    model.SALES_REVENUE = (
        170 * model.W["WHEAT"] + 150 * model.W["CORN"]
        + 36 * model.W["BEETS_FAVORABLE"] + 10 * model.W["BEETS_UNFAVORABLE"]
    )
    model.OBJ = pyo.Objective(
        expr=model.PLANTING_COST + model.PURCHASE_COST - model.SALES_REVENUE,
        sense=pyo.minimize
    )

    # Constraints
    model.CONSTR= pyo.ConstraintList()

    model.CONSTR.add(pyo.summation(model.X) <= 500)
    model.CONSTR.add(
        yields[0] * model.X["WHEAT"] + model.Y["WHEAT"] - model.W["WHEAT"] >= 200
    )
    model.CONSTR.add(
        yields[1] * model.X["CORN"] + model.Y["CORN"] - model.W["CORN"] >= 240
    )
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE"] - model.W["BEETS_UNFAVORABLE"] >= 0
    )
    model.W["BEETS_FAVORABLE"].setub(6000)

    return model

In [2]:
yields = [2.5, 3, 20]
model = build_model(yields)
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Display the objective value to one decimal place
print(f"{pyo.value(model.OBJ):.1f}")

-118600.0


In [3]:
model.X["BEETS"].value

300.0

In [4]:
import mpisppy.utils.sputils as sputils

def scenario_creator(scenario_name):
    if scenario_name == "good":
        yields = [3, 3.6, 24]
    elif scenario_name == "average":
        yields = [2.5, 3, 20]
    elif scenario_name == "bad":
        yields = [2, 2.4, 16]
    else:
        raise ValueError("Unrecognized scenario name")

    model = build_model(yields)
    sputils.attach_root_node(model, model.PLANTING_COST, [model.X])
    model._mpisppy_probability = 1.0 / 3
    return model

In [5]:
opt = pyo.SolverFactory("glpk",warmstart=True)
results = opt.solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -118600.0
  Upper bound: -118600.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 10
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.014734029769897461
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [6]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "glpk"}
all_scenario_names = ["good", "average", "bad"]
ef = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef.solve_extensive_form()

objval = ef.get_objective_value()
print(f"{objval:.1f}")

[    0.08] Initializing SPBase
-108390.0


In [7]:
soln = ef.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

X[BEETS] 250.0
X[CORN] 80.0
X[WHEAT] 170.0


In [8]:
from mpisppy.opt.ef import ExtensiveForm

all_scenario_names = ["good", "average", "bad"]
bounds = {name: -432000 for name in all_scenario_names}
options = {
    "solver": "glpk",
    #"sp_solver_options" : {"threads" : 1},
    "valid_eta_lb": bounds,
    "max_iter": 10,
}

ef = ExtensiveForm(options, all_scenario_names, scenario_creator)
result = ef.solve_extensive_form()

variables = ef.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

[    0.20] Initializing SPBase
good X[BEETS] 250.0
good X[CORN] 80.0
good X[WHEAT] 170.0
average X[BEETS] 250.0
average X[CORN] 80.0
average X[WHEAT] 170.0
bad X[BEETS] 250.0
bad X[CORN] 80.0
bad X[WHEAT] 170.0


In [9]:
ef.write_tree_solution('solution')


In [10]:
from mpisppy.opt.lshaped import LShapedMethod

all_scenario_names = ["good", "average", "bad"]
bounds = {name: -432000 for name in all_scenario_names}
options = {
    "master_solver": "glpk",
    "sp_solver": "glpk",
    "root_solver": "glpk",
    #"sp_solver_options" : {"threads" : 1},
    "valid_eta_lb": bounds,
    "max_iter": 10,
}

ls = LShapedMethod(options, all_scenario_names, scenario_creator)
result = ls.lshaped_algorithm()

variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

[    0.28] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:    0.12 Time Spent on Last Master:    0.02 Time Spent Generating Last Cut Set:    0.09 Current Objective: -1296000.00
Current Iteration: 3 Time Elapsed:    0.23 Time Spent on Last Master:    0.04 Time Spent Generating Last Cut Set:    0.07 Current Objective: -160000.00
Current Iteration: 4 Time Elapsed:    0.34 Time Spent on Last Master:    0.04 Time Spent Generating Last Cut Set:    0.07 Current Objective: -113750.00
Converged in 4 iterations.
Total Time Elapsed:    0.45 Time Spent on Last Master:    0.04 Time spent verifying second stage:    0.06 Final Objective: -108390.00
good X[BEETS] 249.999999999998
good X[CORN] 80.0000000000075
good X[WHEAT] 169.999999999994
average X[BEETS] 249.999999999998
average X[CORN] 80.0000000000075
average X[WHEAT] 169.999999999994
bad X[BEETS] 249.999999999998
bad X[CORN] 80.0000000000075
bad X[WHEAT] 169.999999999994


In [11]:
variables

{('good', 'X[BEETS]'): 249.999999999998,
 ('good', 'X[CORN]'): 80.0000000000075,
 ('good', 'X[WHEAT]'): 169.999999999994,
 ('average', 'X[BEETS]'): 249.999999999998,
 ('average', 'X[CORN]'): 80.0000000000075,
 ('average', 'X[WHEAT]'): 169.999999999994,
 ('bad', 'X[BEETS]'): 249.999999999998,
 ('bad', 'X[CORN]'): 80.0000000000075,
 ('bad', 'X[WHEAT]'): 169.999999999994}

In [12]:
from mpisppy.opt.ph import PH

options = {
    "solvername": "ipopt",
    "PHIterLimit": 5,
    "defaultPHrho": 10,
    "convthresh": 1e-7,
    "verbose": False,
    "display_progress": False,
    "display_timing": False,
    "iter0_solver_options": dict(),
    "iterk_solver_options": dict(),
}

all_scenario_names = ["good", "average", "bad"]
ph = PH(
    options,
    all_scenario_names,
    scenario_creator,
)

[    0.80] Initializing SPBase
[    0.81] Initializing PHBase


brew install ipopt


In [13]:
ph.ph_main()

[    0.84] Creating solvers
[    0.84] Entering solve loop in PHBase.Iter0
[   11.00] Reached user-specified limit=5 on number of PH iterations


(0.8545040634154965, -107722.02946226574, -inf)

In [14]:
variables = ph.gather_var_values_to_rank0()
for (scenario_name, variable_name) in variables:
    variable_value = variables[scenario_name, variable_name]
    print(scenario_name, variable_name, variable_value)

good X[BEETS] 280.6489744760117
good X[CORN] 85.26131627112602
good X[WHEAT] 134.08971425282763
average X[BEETS] 283.27963301152715
average X[CORN] 79.99999920010936
average X[WHEAT] 136.72037278833426
bad X[BEETS] 280.6489744760117
bad X[CORN] 85.26131627112554
bad X[WHEAT] 134.08971425283008
